# CSC 578D / Data Mining / Fall 2018 / University of Victoria

## Python Notebook for Final Project

### The datasets for this project are the following:
1. Crime statistics for the city of Chicago from 2017. [Link to dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2017/d62x-nvdr).
    1. [Download here](http://www.apkc.net/data/csc_578d/project/Chicago_Crimes_2017.csv.zip)
1. The Police beats boundaries for the city of Chicago. [Link to dataset](https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Beats-current-/aerh-rz74).
    1. [Download here](http://www.apkc.net/data/csc_578d/project/Chicago_Boundaries.geojson.zip)
1. List of crimes by special code, in this case named IUCR (Illinois Uniform Crime Reporting). [Link to dataset](https://data.cityofchicago.org/Public-Safety/Chicago-Police-Department-Illinois-Uniform-Crime-R/c7ck-438e/data).
    1. [Download here](http://www.apkc.net/data/csc_578d/project/Chicago_Police_IUCR_Codes.csv.zip)

### Goals of this analysis:
1. Look for patterns in crime.
1. Visualize crime in a concise and efficient manner.
1. Analyze crime as Time Series.
1. Superpose the data onto a map to find better patterns.
1. Build a **Ball tree** of some types of crime segmented by time in intervals of 1 hour. **Ball trees** can be queried very fast to determine if a person is in danger of being victim of a crime based on his/her location and time. Instead of using the Euclidean distance, I will use the Haversine distance, this will help me query the tree based on GPS coordinates of a person. The following is a list of crimes to be included in the tree:
    1. Homicide: Code 1XX
    1. Sexual Assault: Code 2XX
    1. Robbery: Code 3XX
    1. Battery: Code 4XX
    1. Assault: Code 5XX
    1. Theft: Code 8XX
    1. Motor Vehicle Theft: Code 9XX
    1. Deceptive Practice: Code 11XX

### TODO:
1. Separate violent crimes from non-violent ones.
1. Add more dimensions to the **Ball tree**. Maybe I can add one type of crime per dimension, instead of all crimes in 1D.

**Author:** Andreas P. Koenzen <akoenzen@uvic.ca>

**Version:** 0.1

**Disclaimer:** This analysis was made as a project for the course **CSC 578D**, at the University of Victoria during Fall of 2018. I do not own the data used in this analysis, but the analysis was entirely devised by me, including the code.

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sbn
import json
import re
import datetime

from sklearn.neighbors import BallTree
from ast import literal_eval

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Javascript, IFrame
from IPython.core.display import HTML

## Miscellaneous Configuration:

In [ ]:
HTML(
    '<style>'
    'ol           {counter-reset:item}'
    'ol li        {display:block}'
    'ol li:before {content:counter(item) ". ";counter-increment:item;font-weight:bold}'
    'iframe       {border:0px;}'
    'table        {float:left;}'
    '</style>'
)

plt.style.use(['default', 'ggplot'])
plt.rcParams.update({'font.size': 8})

## Environment Variables and Constants:

In [ ]:
# must be set to replicate
data_set_home = %env DATA_SETS_HOME
mapbox_token  = %env MAPBOX_TOKEN

# the earth's radius in kilometers
KMS_PER_RADIAN = 6371.0088

## Functions:

In [ ]:
# function which returns the hours of daylight
# given the day of the year, from 0 to 365
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """
    Compute the hours of daylight for the given date.
    
    :param date:     A pandas' date object.
    :param axis:     Earth's tilt.
    :param latitude: The latitude of the location for which to compute
                     the daylight hours.
                     
    :returns: A scalar value denoting the amount of daylight hours.
    """
    diff = date - pd.datetime(2000, 12, 21)
    day = diff.total_seconds() / 24. / 3600
    day %= 365.25
    m = 1. - np.tan(np.radians(latitude)) * np.tan(np.radians(axis) * np.cos(day * np.pi / 182.625))
    m = max(0, min(m, 2))
    
    return 24. * np.degrees(np.arccos(1 - m)) / 180.

# function which returns if a person is in danger or not of being victim of a crime
# based on their current time of the day and location within the boundaries
# of the city of Chicago.
def am_i_in_danger(tree_list, time: datetime.time, coordinates=(41.841832, -87.623177), radius=0.5):
    """
    Query a Ball tree previously train to compute crimes within a fixed radius,
    and if enough points are returned, according to a constant, then we can resolve
    that that person *MAY* be in danger.
    
    Definition of danger: If the count of crimes within 500 meters of the location
    exceeds or is equals to 10 then it *MAY* be a danger zone.
    
    :param tree_list:   A list of Ball trees. Each index corresponds to 1 hours, starting at midnight.
    :param time:        The time of the day as a datetime.time object.
    :param coordinates: A tuple containing the latitude and longitude.
    :param radius:      The radius in kilometers.
    
    : returns: True if the person is in danger, False otherwise.
    """   
    return 'Yes, you may be in danger.' if np.asscalar(tree_list[time.hour].query_radius(
        np.radians(coordinates).reshape(1, -1), 
        r=(radius / KMS_PER_RADIAN), 
        count_only=True
    ) >= 10) else 'No danger around here, but depends on your luck :)'

## Load the entire dataset into memory as a DataFrame:
1. Index the dataset by the **Date** column.
1. One row corresponds to **one** crime.

In [ ]:
raw_data = pd.read_csv(
    "{0}/CSC_578D/Project/Chicago_Crimes_2017.csv".format(data_set_home),
    index_col=['Date'],
    parse_dates=True,
    date_parser=lambda x: pd.datetime.strptime(x, "%m/%d/%Y %I:%M:%S %p")
)
raw_data.sort_index(inplace=True)

# print basic information about the data
print('Shape: {}'.format(raw_data.shape))
print('Size in memory: {} MB'.format(int(raw_data.memory_usage(index=True, deep=True).sum() / 1024 ** 2)))
raw_data.head()

## Pre-processing:
1. Since the dataset is large in size, I need to remove some unused or duplicate columns. The following are redundant columns, which will be removed.
    1. ID
    1. Case Number
    1. Block: We can remove the block because we will use other information to pin point locations, like Beat, etc.
    1. Primary Type: I will use the IUCR to categorize danger.
    1. Location Description: I will use the "Domestic" column to filter out crimes commited indoors.
    1. Arrest
    1. Community Area
    1. Ward
    1. Year
    1. FBI Code
    1. Updated On
    1. X Coordinate
    1. Y Coordinate
1. Index the entire dataset by the Date column.

In [ ]:
# filter out redundant columns
data = raw_data.drop([
    'ID', 
    'Case Number',
    'Block',
    'Primary Type',
    'Location Description',
    'Arrest',
    'Community Area',
    'Ward',
    'FBI Code',
    'Updated On',
    'X Coordinate',
    'Y Coordinate'
], axis=1)
data.head()

# filtered data
non_domestic_crime = data.loc[(data['Domestic'] == False)]
domestic_crime = data.loc[(data['Domestic'] == True)]

## Describe the dataset:
Gather metrics for:
1. Top 10 Most Violent Beats: In police terminology, a beat is the territory and time that a police officer patrols.
1. Describe the dataset using Time Series analysis.

### Top 10 Most Violent Beats:

In [ ]:
tmp = data.groupby('Beat').size().sort_values(ascending=False)[0:10]
top_10_mv_beats = pd.DataFrame(index=range(1, 11), data={
    'Beat #': tmp.index,
    'Crime Count ' + str(data['Year'].unique()[0]): tmp.values
})
top_10_mv_beats

#### Result:
The most violent **Police Beat** in the city of Chicago is **Beat #1834**. Later on we would see more about the crimes that are commited in this particular Beat and others as well.

### Segment beats into 5 crime categories using the count of crimes for each beat:

The color for each category in the [Choropleth](https://en.wikipedia.org/wiki/Choropleth_map), were drawn from [this](http://colorbrewer2.org/#type=sequential&scheme=Reds&n=5) website.

| ----- | Category 1 | Category 2 | Category 3 | Category 4 | Category 5 |
|-------|------------|------------|------------| -----------|------------|
| Range | 0-600      | 601-1200   | 1201-1800  | 1801-2400  | 2401-10000 |
| Color | #fee5d9    | #fcae91    | #fb6a4a    | #de2d26    | #a50f15    |

In [ ]:
beats_sorted = non_domestic_crime.groupby('Beat').size().sort_values(ascending=False)

categories = []
for i in range(0, 5):
    categories.append(beats_sorted[(beats_sorted > (i * 600)) & (beats_sorted <= (10000 if (i == 4) else (i * 600) + 600))])
for e in categories:
    pd.DataFrame(data=e.head(2), columns=['Crime Count']) # pretty print the first 2 of each

### Group data by crime type and keep the top 10:
Filter both by Beat and IUCR. We should have a multi-index DataFrame, where the first index is **Beat** and the second will be **IUCR**, then we would have only one column named **Crime Count** listing the count of crimes. Then finally we should join our DataFrame with the one built from the **Chicago_Police_IUCR_Codes.csv** file and add both description columns from the latter.

In [ ]:
# use Pandas Hierarchical Indexing
tmp = non_domestic_crime.groupby(['Beat', 'IUCR']).size()
tmp = tmp.sort_values(0, ascending=False).sort_index(level='Beat', sort_remaining=False)
tmp = tmp.groupby(level=0).head(10) # list the top 10 for each Beat
tmp = pd.DataFrame(index=tmp.index, data={'Crime Count': tmp})
# tmp.head()

# now match each IUCR with a description from Chicago_Police_IUCR_Codes.csv file
iucr_codes = pd.read_csv(
    "{0}/CSC_578D/Project/Chicago_Police_IUCR_Codes.csv".format(data_set_home)
)
iucr_codes['IUCR'] = iucr_codes['IUCR'].apply(lambda x: x.zfill(4)) # pad with leading 0 (zero) if IUCR entry is less than 4
iucr_codes.set_index('IUCR', inplace=True)
# iucr_codes.head()

# add new column with the secondary description
# see: http://blog.rlucas.net/bugfix/pandas-merge-woes-on-multiindex-solved/
tmp.index.levels[0].name = 'Beat'
tmp.index.levels[1].name = 'IUCR'
iucr_codes.index.name = 'IUCR'
tmp = tmp.join(iucr_codes) # magic! no need to specify the merging key!

# drop innecessary columns
tmp = tmp.drop(['INDEX CODE'], axis=1)
tmp.columns = [
    'Crime Count', 
    'Primary', 
    'Secondary'
]

crime_by_beat_iucr = tmp
tmp.head(20) # pretty print the first 2 beats

### Time Series analysis:
**Requisite:** The dataset needs to be indexed by date.

#### Plot data by month:

In [ ]:
# build a new index for months
months = pd.period_range('2017-01', periods=12, freq='M')

month = data.groupby(data.index.month).size()
month = pd.DataFrame(index=months, data={'Crimes': list(month)})

fig, ax = plt.subplots(figsize=(6, 4))
_ = month.plot(
    ax=ax,
    title='Crimes by Month',
    style=['--']
)
_ = ax.set_ylabel('Count')
_ = ax.set_xlabel('Month')

#### Result:
Given the chart above we can see that a pattern emerges from the data, where crimes are commited more often during summer months. So I will formulate an hypothesis and look further to see if it holds.

#### Hypothesis 1:
Crimes are higher during summer months, due to more people being outdoors than indoors.

#### Hypothesis 1 Testing:
To test this hypothesis I will separate domestic crimes from non-domestic crimes and see if the pattern is still present.

In [ ]:
months = pd.period_range('2017-01', periods=12, freq='M')

month_outdoor = non_domestic_crime.groupby(non_domestic_crime.index.month).size()
month_outdoor = pd.DataFrame(index=months, data={'Crimes': list(month_outdoor)})

month_indoor = domestic_crime.groupby(domestic_crime.index.month).size()
month_indoor = pd.DataFrame(index=months, data={'Crimes': list(month_indoor)})

fig, (ax_left, ax_right) = plt.subplots(ncols=2, figsize=(12, 4))
_ = ax_left.set_ylabel('Count')
_ = ax_left.set_xlabel('Month')
_ = ax_right.set_ylabel('Count')
_ = ax_right.set_xlabel('Month')

_ = month_outdoor.plot(
    ax=ax_left,
    title='Non-Domestic Crimes by Month',
    style=['--']
)
_ = ax_left.legend(["{} Crimes Total".format(month_outdoor['Crimes'].sum())])

_ = month_indoor.plot(
    ax=ax_right,
    title='Domestic Crimes by Month',
    style=['--']
)
_ = ax_right.legend(["{} Crimes Total".format(month_indoor['Crimes'].sum())])

#### Result:
Well, we can see after separating the two sets (outdoor & indoor) crimes, that hypothesis 1 is rejected since both charts follow the same pattern. We can see in the right chart that indoor crimes also spike during summer months. Maybe we can formulate another hypothesis for why this pattern occurs.

#### Hypothesis 2:
Non-domestic crimes are higher during summer months, due to more daylight hours, and more people are in the streets.

#### Hypothesis 2 Testing:

In [ ]:
hour = non_domestic_crime.groupby(non_domestic_crime.index.hour).size()
hour = pd.DataFrame(index=hour.index, data={'Crimes': list(hour)})
# hour

fig, (ax_left, ax_right) = plt.subplots(ncols=2, figsize=(12, 4))
_ = hour.plot(
    ax=ax_left,
    title='Crimes by Hour',
    style=['--']
)
_ = ax_left.set_ylabel('Count')
_ = ax_left.set_xlabel('Hour')

start = datetime.datetime(year=2017, month=1, day=1)
days = start + pd.to_timedelta(np.arange(365), "D")

# iterate all days an compute the daylight hours
hours = pd.DataFrame(
    data=days.map(lambda day: hours_of_daylight(day, latitude=41.841832)),
    columns=["Daylight Hours"],
    index=days
)
_ = hours.plot(
    ax=ax_right,
    title='Daylight Hours Chicago',
    style=['--'],
    color='b'
)
_ = ax_right.set_ylabel('Hours of Daylight')
_ = ax_right.set_xlabel('Month')

I have plotted Chicago crimes by hour on the left, and on the right we have a plot of daylight hours by month. We can observe that crime **rises** at 5:00AM, increases during the day, up to a peak at noon and later starts to decrease roughly at 6:00PM. This implies that crimes happen more often during daylight hours. That **may** be a reason why we observed more crimes during the summer months than winter months, but that is subject to much more analysis and variables like temperature and others.

#### Weekdays vs. Weekends:

In [ ]:
compound = non_domestic_crime.groupby([
    np.where(non_domestic_crime.index.weekday < 5, "Weekday", "Weekend"), 
    non_domestic_crime.index.hour
]).size()
# compound

fig, (ax_left, ax_right) = plt.subplots(ncols=2, figsize=(12, 4))
_ = compound['Weekday'].plot(
    ax=ax_left,
    title='Crimes during Weekdays',
    style=['--']
)
_ = ax_left.set_ylabel('Count')
_ = ax_left.set_xlabel('Time')

_ = compound['Weekend'].plot(
    ax=ax_right,
    title='Crimes during Weekends',
    style=['--']
)
_ = ax_right.set_ylabel('Count')
_ = ax_right.set_xlabel('Time')

#### Results:
Above I plotted crimes segmented by **Weekdays** and **Weekends**, trying to further the analysis, but no distinctive patterns emerged from this last analysis, besides that crime **rises** a few minutes later on weekends. So I can say that more analysis is need to test *Hypothesis 2*. Maybe segment crimes by their type before doing the time series analysis.

## Visualization:
1. Plot crimes by Beat.
1. Build a Choropleth using the Chicago Boundaries dataset.

In [ ]:
ax = top_10_mv_beats.plot.bar(
    x=0,
    y=1,
    figsize=(6, 4),
    title='Top 10 Crimes Police Beats',
    color='r'
);
ax.set_ylabel('Count');
ax.set_xlabel('Beat #');
ax.set_axisbelow(True);
ax.yaxis.grid(color='gray', linestyle='dashed');
ax.xaxis.grid(color='gray', linestyle='dashed');

## Choropleth:
1. First separate the map into beats covering the entire city of Chicago.
1. Plot the data using a choropleth chart.
1. Filter crime by outdoor only.
1. Filter crime by top 10 crimes per beat.

In [ ]:
# read the boundaries dataset
boundaries = ''
with open("{0}/CSC_578D/Project/Chicago_Boundaries.geojson".format(data_set_home)) as json_file:  
    boundaries = json.load(json_file)

In [ ]:
# build the multipolygons layers
layers = ''
layer = """
map.addLayer({{
  'id':'Beat_{}_{}',
  'type':'{}',
  'source':{{
    'type':'geojson',
    'data':{{
      'type':'Feature',
      'geometry':{{
        'type':'Polygon',
        'coordinates':{}
      }}
    }}
  }},
  'layout': {{}},
  'paint': {{
    '{}':'{}',
    '{}':{}
  }}
}});
"""
popups = ''
popup = """
map.on('click', 'Beat_{}_Fill', function (e) {{
    new mapboxgl.Popup().setLngLat(e.lngLat).setHTML('{}').addTo(map);
}});
map.on('mouseenter', 'Beat_{}_Fill', function () {{
    map.getCanvas().style.cursor = 'pointer';
}});
map.on('mouseleave', 'Beat_{}_Fill', function () {{
    map.getCanvas().style.cursor = '';
}});
"""
for _, e in enumerate(boundaries['features']):
    beat_num = e['properties']['beat_num']
    coordinates = e['geometry']['coordinates'][0]
    crimes = 0
    
    if int(beat_num) == 3100:
        continue
    
    # use a different color for each category
    colors = ['#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
    color = ''
    for i, e in enumerate(categories):
        crimes = e.get(int(beat_num), default=None)
        if crimes != None:
            color = colors[i]
            break
            
    # get beat information
    beat_info = '<ol>'
    for index, data in crime_by_beat_iucr.iterrows():
        if int(index[0]) == int(beat_num):
            beat_info += '<li>{}-{} ({} crimes)</li>'.format(data[1], data[2], data[0])
    beat_info += '</ol>'
    
    layers += layer.format(
        beat_num, 
        'Fill',
        'fill',
        json.dumps(coordinates),
        'fill-color',
        color if color != '' else '#999',
        'fill-opacity',
        0.75
    )
    layers += layer.format(
        beat_num, 
        'Outline',
        'line',
        json.dumps(coordinates), 
        'line-color', 
        '#67000d', 
        'line-width', 
        0.60
    )
    popup_html = "<h1>Beat ID: {}</h1><p>Crimes #: {} crimes in 2017</p>{}".format(
        beat_num, 
        crimes,
        beat_info
    )
    popups += popup.format(
        beat_num,
        popup_html,
        beat_num,
        beat_num
    )

In [ ]:
replacements = {
    'mapbox_token': mapbox_token,
    'layers': layers,
    'popups': popups,
    'total_crimes': str(beats_sorted.sum())
}

html = """
<!DOCTYPE html>
<html>
<head>
    <meta charset='utf-8'/>
    <title>Project</title>
    <meta name='viewport' content='initial-scale=1,maximum-scale=1,user-scalable=no'/>
    <script src='https://api.tiles.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.js'></script>
    <link href='https://api.tiles.mapbox.com/mapbox-gl-js/v0.51.0/mapbox-gl.css' rel='stylesheet'/>
    <style>
    body {margin:0; padding:0;}
    #map {position:absolute;top:0;bottom:0;width:100%;}
    .legend {
        background-color:#fff;
        border-radius:3px;
        bottom:30px;
        box-shadow:0 1px 2px rgba(0,0,0,0.10);
        font:12px/20px 'Helvetica Neue', Arial, Helvetica, sans-serif;
        padding:10px;
        position:absolute;
        right:10px;
        z-index:1;
    }
    .legend h4 {margin:0 0 10px;}
    .legend div span {
        border-radius:50%;
        display:inline-block;
        height:10px;
        margin-right:5px;
        width:10px;
    }
    .mapboxgl-popup {max-width:400px;font:12px/20px 'Helvetica Neue', Arial, Helvetica, sans-serif;}
    </style>
</head>
<body>
    <div id='map'></div>
    <div id='crime-legend' class='legend'>
    <h4>Chicago Non-Domestic Crimes by Police Beat per Year (2017)</h4>
    <div><span style='background-color:transparent'></span>No Data</div>
    <div><span style='background-color:#fee5d9'></span>0 - 600 crimes</div>
    <div><span style='background-color:#fcae91'></span>601 - 1200</div>
    <div><span style='background-color:#fb6a4a'></span>1201 - 1800</div>
    <div><span style='background-color:#de2d26'></span>1801 - 2400</div>
    <div><span style='background-color:#a50f15'></span>2401 - 10000</div>
    <h4>Total Non-Domestic Crimes: (total_crimes)</h4>
</div>
    <script>
    mapboxgl.accessToken = '(mapbox_token)';
    var map = new mapboxgl.Map({
        container: 'map',
        style: 'mapbox://styles/mapbox/streets-v10',
        center: [-87.623177, 41.841832],
        zoom: 9.5
    });
    map.on('load', function () {
        (layers)
        (popups)
    });
    </script>
</body>
</html>
"""

for key, value in replacements.items():
    html = html.replace("({})".format(key), value)

display(HTML('<iframe srcdoc="{srcdoc}" style="width:100%;height:800px;"></iframe>'.format(srcdoc=html)))

## Segment and plot some crimes as scatter plots:
This will help to detect patterns, specially if segmented by hour.

In [ ]:
tmp = non_domestic_crime.copy()
# tmp.head()

# filter by robbery. in the IUCR list, robbery' codes start with 3!
tmp['IUCR'] = tmp['IUCR'].apply(lambda x: x.zfill(4))
tmp = tmp[tmp.apply(lambda x: str(x['IUCR']).startswith((
    '01',
    '02',
    '03',
    '04',
    '05',
    '08',
    '09',
    '011'
)), axis=1)]

robbery_data = tmp
# robbery_data.head()

fig, ax = plt.subplots(ncols=2, nrows=6, figsize=(16, 40), sharex=True, sharey=True)
s = 0
for k in range(6):
    e = s + 2
    
    d = robbery_data.between_time(
        '{}:00'.format(s), 
        '{}:00'.format(e if e != 24 else '23:59')
    )
    
    _ = d.plot.scatter(
        x='Longitude', 
        y='Latitude',
        ax=ax[k][0],
        title='Crimes (From {}hs to {}hs)'.format(
            '{}:00'.format(s), 
            '{}:00'.format(e if e != 24 else '23:59')
        ),
        s=2
    )
    
    s = e
    e = e + 2
    
    d = robbery_data.between_time(
        '{}:00'.format(s), 
        '{}:00'.format(e if e != 24 else '23:59')
    )
    
    _ = d.plot.scatter(
        x='Longitude', 
        y='Latitude',
        ax=ax[k][1],
        title='Crimes (From {}hs to {}hs)'.format(
            '{}:00'.format(s), 
            '{}:00'.format(e if e != 24 else '23:59')
        ),
        s=2
    )
    
    s = e

## k-Nearest Neighbor:
1. Build a **Ball tree** for each time segment.

In [ ]:
s = [re.sub('^\d days ', '', str(x)) for x in pd.timedelta_range(start='00:00:00', end='24:00:00', freq='1h', closed='left')]
e = [re.sub('^\d days ', '', str(x - pd.Timedelta(minutes=1))) for x in pd.timedelta_range(start='00:00:00', end='24:00:00', freq='1h', closed='right')]

segmented_robberies = []
for k in range(24):
    segmented_robberies.append(list(robbery_data.between_time(s[k], e[k])['Location'].dropna()))
    
trees = []
for e in segmented_robberies:
    locations = []
    for k in e:
        locations.append(list(literal_eval(k)))
    trees.append(BallTree(np.radians(locations), leaf_size=30, metric='haversine'))

## Inferences:

In [ ]:
HTML('<img src="http://www.apkc.net/img/images/CSC_578D_1.png"/>')
am_i_in_danger(
    tree_list=trees, 
    time=datetime.time(13, 10, 20), 
    coordinates=(41.900889, -87.709259)
)

HTML('<img src="http://www.apkc.net/img/images/CSC_578D_2.png">')
am_i_in_danger(
    tree_list=trees, 
    time=datetime.time(13, 10, 20), 
    coordinates=(41.973190, -87.839996)
)

***
# The End